<p style="font-size: 35px; color: violet; text-align: center;">JIRA</p>

* Dependencias: 
    * Todas las bibliotecas de python en el bloque siguiente 
    * Chrome drive

In [16]:
import numpy as np
import pandas as pd 
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


from bs4 import BeautifulSoup

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

#Execute shell commands
import subprocess

---
---

## Functions

In [17]:
def petition_html(petition):
    #Open the petition
    driver.execute_script(f"window.open('https://jira.globaldevtools.bbva.com/browse/{petition}')")
    tabs = driver.window_handles
    driver.switch_to.window(tabs[-1])

    html = driver.page_source
    
    description = driver.find_elements(By.XPATH, "//div[@class='user-content-block']")
    descriptions = []
    for desc in description:
        descriptions.append(desc.get_attribute('innerHTML'))
    
    
    driver.switch_to.window(tabs[0])
    time.sleep(2)
    return descriptions

---

### Google sheets connection

In [18]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"

#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com
scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1v9wCFg6f6geoiiknvNUWjxd3XxIFLJb8aMUfRcTWKAc"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    # Seleccionar la hoja específica por nombre
    sheet_name = "Octubre"  # Reemplaza con el nombre de tu hoja
    sheet = spreadsheet.worksheet(sheet_name)
    # Obtener los encabezados esperados
    expected_headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]

    # Obtener todos los registros de la hoja
    data = sheet.get_all_records(expected_headers=expected_headers)
    df = pd.DataFrame(data)

    # Mostrar las primeras filas del DataFrame
    print(df.head())

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Octubre', 'Septiembre', 'Responsables', 'Extra', 'Copia de 2024', 'Semana 1010', 'Tareas']
  Fecha de alta Fecha incurrida Fecha de fin UUAAA        Código SDATOOL  \
0    24/09/2024      01/10/2024   01/10/2024  KESJ  DQ-DP-007019   45403   
1    24/09/2024      01/10/2024   01/10/2024  KESJ  DQ-DP-007019   45403   
2    27/09/2024      01/10/2024   01/10/2024  FACC  DQ-DP-007037   34644   
3    27/09/2024      01/10/2024   01/10/2024  KHFC                         
4    30/09/2024      01/10/2024   01/10/2024  MCRR  DQ-DP-007041   38358   

          Feature         Petición Geografía     Gestor BBDD   Ámbito  \
0  AUDLEGGES4-249  REQ000101515343    España       PosgreSQL   Legacy   
1  AUDLEGGES4-249  REQ000101515343    España       PosgreSQL   Legacy   
2         ADH-678     OTRSSD-17407    España  Oracle Physics  NextGEN   
3                     OTRSSD-17408            Oracle Physi

<p style="font-size: 25px; color: #abebc6;">Limpiamos el Data Frame proveniente del 'excel'</p>

In [19]:
# Eliminar filas vacías y columnas que no van
df = df[df["Fecha de alta"] != ""]
try: 
    df.drop(columns=[""], inplace=True)
except KeyError:
    pass
# Convertir las fechas a formato datetime
df[["Fecha de alta", "Fecha incurrida", "Fecha de fin"]] = df[["Fecha de alta", "Fecha incurrida", "Fecha de fin"]].apply(pd.to_datetime, format="%d/%m/%Y")

# Reemplazar celdas vacías por None
df.replace("", None, inplace=True)

df

,Fecha de alta,Fecha incurrida,Fecha de fin,UUAAA,Código,SDATOOL,Feature,Petición,Geografía,Gestor BBDD,Ámbito,Responsable,Validada,Horas,Comentarios,Estado
0,2024-09-24,2024-10-01,2024-10-01,KESJ,DQ-DP-007019,45403,AUDLEGGES4-249,REQ000101515343,España,PosgreSQL,Legacy,David,Si,3,"Pendiente DROP, volver a hacer check-in",Cerrado
1,2024-09-24,2024-10-01,2024-10-01,KESJ,DQ-DP-007019,45403,AUDLEGGES4-249,REQ000101515343,España,PosgreSQL,Legacy,Matías,No,3,"Pendiente DROP, volver a hacer check-in",Cerrado
2,2024-09-27,2024-10-01,2024-10-01,FACC,DQ-DP-007037,34644,ADH-678,OTRSSD-17407,España,Oracle Physics,NextGEN,David,Si,0.5,None,Cerrado
3,2024-09-27,2024-10-01,2024-10-01,KHFC,None,None,None,OTRSSD-17408,None,Oracle Physics,NextGEN,Pedro,No,1,Descartada,Cerrado
4,2024-09-30,2024-10-01,2024-10-01,MCRR,DQ-DP-007041,38358,ADH-690,OTRSSD-17410,Global,Oracle Physics,NextGEN,David,Si,1.5,None,Cerrado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2024-10-24,NaT,NaT,XBOB,None,44974,WSONEPIV13-1593,OTRSSD-17798,Global,Oracle Physics,NextGEN,None,None,12/11,None,Backlog
168,2024-10-24,NaT,NaT,EKSA,None,45266,SISIF-806,OTRSSD-17799,España,Oracle Physics,NextGEN,None,None,12/11,None,Backlog
169,2024-10-24,NaT,NaT,IMPT,None,449784,WSONEPIV19-1467,OTRSSD-17802,España,Oracle Physics,NextGEN,None,None,12/11,None,Backlog
170,2024-10-24,NaT,NaT,H9IY,None,45905,EXTSERV-376,OTRSSD-17803,Global,Oracle Physics,NextGEN,None,None,None,None,Backlog


---

In [20]:
import socket

def check_port_in_use(port, host='127.0.0.1'):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex((host, port)) == 0

port = 9222
host = '127.0.0.1'
if check_port_in_use(port, host):
    print(f"El puerto {port} está en uso en {host}.")
else:
    print(f"El puerto {port} no está en uso en {host}.")

El puerto 9222 no está en uso en 127.0.0.1.


## Chrome connection Windows

In [21]:
#Ejecutar el comando en el cmd
command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'
process = subprocess.Popen(command, shell=True)
time.sleep(2)

#Verifield if the port 9222 is open
command_check = "netstat -ano | findstr 9222"
process_check = subprocess.Popen(command_check, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process_check.communicate()
if stdout:
    print("El puerto 9222 está en uso:")
    print(stdout.decode())
else:
    print("El puerto 9222 no está en uso o hubo un error:")
    print(stderr.decode())

#Configurar las opciones de Chrome para conectarse a la instancia Chrome existente
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#Configuración del controlador de Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

html = driver.page_source

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\matia\AppData\Local\Temp\ipykernel_8668\980132825.py:2: SyntaxWarning: invalid escape sequence '\P'
  command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'


El puerto 9222 está en uso:
  TCP    127.0.0.1:9222         0.0.0.0:0              LISTENING       11232
  TCP    127.0.0.1:9222         127.0.0.1:57351        FIN_WAIT_2      14480
  TCP    127.0.0.1:9222         127.0.0.1:57352        TIME_WAIT       0
  TCP    127.0.0.1:57351        127.0.0.1:9222         CLOSE_WAIT      22408



---

<p style="text-align: center; font-size: 35px; color: skyblue;">Going into JIRA</p>

### Getting JIRA info

### Header

In [22]:
#Execute the code in this URL: https://jira.globaldevtools.bbva.com/secure/RapidBoard.jspa?rapidView=33300&quickFilter=106472

driver.get("https://jira.globaldevtools.bbva.com/secure/RapidBoard.jspa?rapidView=33300&quickFilter=106472")
time.sleep(5)
#Get from HTML WEB
element_header = driver.find_element(By.XPATH, "//ul[@class='ghx-column-headers']")
element_header = element_header.get_attribute('innerHTML')

#Starting the local
element_haux = element_header.split("<li")

start_id = element_haux[2].find("id=")
end_id = start_id + 11

list_header = ["To Do", "Analysing", "Pending to information", "In progress", "Ready to deploy", "Ready to verify", "Done"]
list_header_ids = []
for i in element_haux[1: ]:
    list_header_ids.append(i[start_id: end_id])

list_headers = dict(zip(list_header, list_header_ids))
print(list_headers)

{'To Do': 'id="281906"', 'Analysing': 'id="281911"', 'Pending to information': 'id="281914"', 'In progress': 'id="281907"', 'Ready to deploy': 'id="282052"', 'Ready to verify': 'id="282060"', 'Done': 'id="281908"'}


### Columns information

In [23]:
days = range(30) 
xpaths = [f"//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-{day} ghx-type-10900 ui-draggable ui-draggable-handle']" for day in days]
combined_xpath = "|".join(xpaths)

# Geting the petition tag
elements = driver.find_elements(By.XPATH, combined_xpath)
petitions_code = []
for element in elements: 
    petitions_code.append(element.get_attribute('innerHTML'))

for i in range(len(petitions_code)): 
    petitions_code[i] = petitions_code[i][99: 111]

print(len(petitions_code), petitions_code)

62 ['OTRSSD-17826', 'OTRSSD-17811', 'OTRSSD-17809', 'OTRSSD-17808', 'OTRSSD-17807', 'OTRSSD-17806', 'OTRSSD-17805', 'OTRSSD-17804', 'OTRSSD-17803', 'OTRSSD-17802', 'OTRSSD-17799', 'OTRSSD-17798', 'OTRSSD-17787', 'OTRSSD-17784', 'OTRSSD-17782', 'OTRSSD-17781', 'OTRSSD-17778', 'OTRSSD-17777', 'OTRSSD-17754', 'OTRSSD-17753', 'OTRSSD-17737', 'OTRSSD-17736', 'OTRSSD-17735', 'OTRSSD-17713', 'OTRSSD-17694', 'OTRSSD-17693', 'OTRSSD-17689', 'OTRSSD-17686', 'OTRSSD-17684', 'OTRSSD-17681', 'OTRSSD-17680', 'OTRSSD-17679', 'OTRSSD-17678', 'OTRSSD-17668', 'OTRSSD-17667', 'OTRSSD-17666', 'OTRSSD-17665', 'OTRSSD-17664', 'OTRSSD-17663', 'OTRSSD-17662', 'OTRSSD-17661', 'OTRSSD-17660', 'OTRSSD-17659', 'OTRSSD-17657', 'OTRSSD-17649', 'OTRSSD-17647', 'OTRSSD-17646', 'OTRSSD-17645', 'OTRSSD-17644', 'OTRSSD-17643', 'OTRSSD-17642', 'OTRSSD-17641', 'OTRSSD-17630', 'OTRSSD-17620', 'OTRSSD-17619', 'OTRSSD-17616', 'OTRSSD-17615', 'OTRSSD-17614', 'OTRSSD-17613', 'OTRSSD-17597', 'OTRSSD-17510', 'OTRSSD-17508']


---

<p style="text-align: center; font-size: 35px; color: #42bf21;">Google sheets</p>

In [24]:
print(df.columns)
print(petitions_code)

Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado'],
      dtype='object')
['OTRSSD-17826', 'OTRSSD-17811', 'OTRSSD-17809', 'OTRSSD-17808', 'OTRSSD-17807', 'OTRSSD-17806', 'OTRSSD-17805', 'OTRSSD-17804', 'OTRSSD-17803', 'OTRSSD-17802', 'OTRSSD-17799', 'OTRSSD-17798', 'OTRSSD-17787', 'OTRSSD-17784', 'OTRSSD-17782', 'OTRSSD-17781', 'OTRSSD-17778', 'OTRSSD-17777', 'OTRSSD-17754', 'OTRSSD-17753', 'OTRSSD-17737', 'OTRSSD-17736', 'OTRSSD-17735', 'OTRSSD-17713', 'OTRSSD-17694', 'OTRSSD-17693', 'OTRSSD-17689', 'OTRSSD-17686', 'OTRSSD-17684', 'OTRSSD-17681', 'OTRSSD-17680', 'OTRSSD-17679', 'OTRSSD-17678', 'OTRSSD-17668', 'OTRSSD-17667', 'OTRSSD-17666', 'OTRSSD-17665', 'OTRSSD-17664', 'OTRSSD-17663', 'OTRSSD-17662', 'OTRSSD-17661', 'OTRSSD-17660', 'OTRSSD-17659', 'OTRSSD-17657', 'OTRSSD-17649', 'OTRSSD-17647', 'OTRSSD-17646',

#### Obtener peticiones que no están en 'excel'

In [25]:
petitions_for_add = []
for i in petitions_code:
    if i not in df["Petición"].values:
        petitions_for_add.append(i)

print(petitions_for_add)

['OTRSSD-17826', 'OTRSSD-17811', 'OTRSSD-17809', 'OTRSSD-17808', 'OTRSSD-17807', 'OTRSSD-17806', 'OTRSSD-17805']


#### Info de las peticiones faltanes

#### Explotando dentro de las peticiones [Ir a la sección de funciones](#functions)

In [26]:
for petition in petitions_for_add:
    description = petition_html(petition)
    print(description)
    
    # Limpiando las descripciones
    replacements = {
        "<p>": "", "</p>": "",
        "\n": "", 
        "<br>": "",
        "<div>": "", "</div>": "",
        "<span>": "", "</span>": "",
        "<strong>": "", "</strong>": "",
        "<a>": "", "</a>": "", 
        "&nbsp;": "", 
        "/t": "", 
        "<li>": "", "</li>": ""
    }
    for old, new in replacements.items():
        description = description.replace(old, new)

NameError: name 'desc' is not defined

In [13]:
current_url = driver.current_url
page_title = driver.title
print(f"Current URL: {current_url}")
print(f"Page Title: {page_title}")

Current URL: https://jira.globaldevtools.bbva.com/secure/RapidBoard.jspa?rapidView=33300&quickFilter=106472
Page Title: Data&Process_SD_GLOBAL - Agile Board - BBVA - JIRA


---
---